### 1.2.2 レイヤとしてのクラス化と順伝播の実装

- Python版ではレイヤをclassで定義していたが，Juliaではclassの概念が存在しない．
  - モデルをコンストラクタで定義し，それにメソッドを作用させていくように考える．
- そのため，レイヤに持たせていたメソッド`forward()`と`backward()`，インスタンス変数`params`と`grads`を独立させる．
  - メソッドは`function`で定義し，インスタンス変数は`struct`に持たせる．
  - Juliaの多重ディスパッチの機構を利用し，同じ名前のメソッドでも型によって異なる処理を呼び出す．
    - レイヤを型として定義する．

In [1]:
using LinearAlgebra

In [2]:
# 抽象型の定義
abstract type Layer end

In [3]:
# Sigmoidレイヤの定義
struct Sigmoid <: Layer
    a
    # コンストラクタメソッド
    Sigmoid() = new()
end

In [4]:
# Affineレイヤの定義
struct Affine <: Layer
    W
    b
end

In [5]:
# Python版のSigmoidクラスのforward()
function forward(x, params::Sigmoid)
    return 1.0 ./ (1.0 .+ exp.(-x))
end

forward (generic function with 1 method)

In [6]:
# Python版のAffineクラスのforward()
function forward(x, params::Affine)
    return x * params.W .+ reshape(params.b, 1, :) # bを1x4 Arrayにreshape
end

forward (generic function with 2 methods)

In [7]:
# TwoLayerNetの定義
mutable struct TwoLayerNet
    I # input_size
    H # hidden_size
    O # output_size
    W1
    b1
    W2
    b2
    layers
    
    # コンストラクタメソッド
    function TwoLayerNet(I, H, O)
        # 重みとバイアスの初期化
        W1 = randn(I, H)
        b1 = randn(H)
        W2 = randn(H, O)
        b2 = randn(O)
        
        # レイヤの生成
        layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        new(I, H, O, W1, b1, W2, b2, layers)
    end
end

In [8]:
# Python版のTwoLayerNetのpredict()
function predict(x, model::TwoLayerNet)
    for layer in model.layers
        x = forward(x, layer)
    end
    return x
end

predict (generic function with 1 method)

In [9]:
x = randn(10, 2)
model = TwoLayerNet(2, 4, 3)
s = predict(x, model)

10×3 Array{Float64,2}:
 -3.53433  -2.31897    0.200002
 -2.3876   -1.27827   -0.192078
 -1.78165  -0.775269   0.0106457
 -1.89293  -0.801865   0.0831063
 -3.39707  -2.21111    0.293754
 -3.51111  -2.29646    0.178192
 -3.10325  -1.38386   -0.956893
 -2.21165  -0.890576  -0.703168
 -3.77391  -1.92597   -0.934131
 -1.82843  -0.802474   0.017584